In [16]:
import math
import torch
from torch import nn
from torch.nn import functional as F
from data_plot import *
from NLP_data_read import *

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size,num_steps=32,35 #num_steps the size of the sequence
train_iter, vocab = load_data_time_machine(batch_size, num_steps)
# vocab：dict类型 整数的index 会转换成对应的word

In [4]:
F.one_hot(b.tensor([0,2]),len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [13]:
X = torch.arange(10).reshape((2, 5))
print(X)
print(F.one_hot(X.T, 28).shape)
print(F.one_hot(X, 28))

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
torch.Size([5, 2, 28])
tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [8]:
def get_params(vocab_size,num_hiddens,device):
    num_inputs=num_outputs=vocab_size
    #一個個詞通過one hot 變成向量之後，就會變成長為vocab size 的向量 ，所以mlp輸入的維度就是vocab的size
    
    def normal(shape):
        return torch.randn(size=shape,device=device)*0.01
        # initialization the parameter with mean=0 and sd=1
        
    W_xh=normal((num_inputs,num_hiddens)) # the weight from input to the hidden layer
    W_hh=normal((num_hiddens,num_hiddens))# the weight from last step of hidden layer to current step
    b_h = torch.zeros(num_hiddens, device=device)

    W_hq=normal((num_hiddens,num_outputs))
    b_q=torch.zeros(num_outputs,device=device)
    params = [W_xh,W_hh,b_h,W_hq,b_q]
    
    for param in params:
        param.requires_grad_(True)
    return params
    
    
    
    
    
    
    

In [9]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

#initialization the hidden variable, because when we at beganing of step 0 ,we do not have perivious hidden variable



In [14]:
def rnn(inputs, state,params):
    #sate: initial hidden variable
    W_xh,W_hh,b_h,W_hq,b_q=params
    
    H,=state
    outputs=[]
    for X in inputs:
        H=torch.tanh(torch.mm(X,W_xh)
                    +torch.mm(H,W_hh)
                    +b_h)
        Y=torch.mm(H,W_hq)+b_q
        outputs.append(Y)
    return torch.cat(outputs,dim=0),(H,)



In [15]:
class RNNModelScratch: 

    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn
                            #forward function
    def __call__(self, X, state):# forward
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [18]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, device, get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], device)
Y, new_state = net(X.to(device), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

In [ ]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    # prefix 给定的开头
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])